In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math
# !pip install transformers
from transformers import pipeline
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv("C:/Users/lasko/Documents/Bocconi/2nd Semester/Natural Language Processing/Final Project/company_aspect_matrix_with_counts.csv")

Company aspect

In [ ]:
df.head()

##### Short EDA for company aspects

In [ ]:
df['n_mentions'].min(), df['n_mentions'].max()

In [ ]:
bins = list(range(0, 110, 5))

plt.figure(figsize=(8,5))
plt.hist(df['n_mentions'], bins=bins, edgecolor='black', align='left')
plt.title('Distribution of Number of Mentions')
plt.xlabel('Number of Mentions')
plt.ylabel('Frequency')
plt.xticks(bins)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In terms of aspects, they are most often mentioned up to 5 times.

What if we were to drop the aspects for which the number of mentions is lower than 3?

In [ ]:
# Count number of aspects per firm
aspect_count = df.groupby('firm')['aspect'].count().reset_index(name='n_aspects')
aspect_count.sort_values('n_aspects', ascending=False)

In [ ]:
print(aspect_count[aspect_count['n_aspects'] < 5])
len(aspect_count[aspect_count['n_aspects'] < 5])

In [ ]:
# Drop rows where n_mentions < 3
filtered_df = df[df['n_mentions'] >= 3]

# Count number of aspects per firm after filtering
aspect_count_filtered = filtered_df.groupby('firm')['aspect'].count().reset_index(name='n_aspects_filtered')
sorted_aspect_count_filtered = aspect_count_filtered.sort_values('n_aspects_filtered', ascending=False)
sorted_aspect_count_filtered[sorted_aspect_count_filtered['n_aspects_filtered'] < 3]


In [ ]:
comparison = aspect_count.merge(aspect_count_filtered, on='firm', how='left')

comparison['n_dropped'] = comparison['n_aspects'] - comparison['n_aspects_filtered']

comparison['n_dropped'].min(), comparison['n_dropped'].max()


**INSIGHTS**:
- In order not to lose too many aspects due to their low number of mentions, we're **discarding the filtering based on n_mentions**
- To maximize the best fit based on aspects, we're **filtering out companies (17) that have less than 5 aspects** mentioned in the reviews.

Company aspect - pivoted

In [ ]:
df_pivot = df.pivot(
    index='firm',
    columns='aspect',
    values='avg_star_rating'
)

In [ ]:
df_pivot.head()

## Summarization - pros & cons

In [ ]:
df_reviews = pd.read_csv("C:/Users/lasko/Documents/Bocconi/2nd Semester/Natural Language Processing/Final Project/comparison_cleans.csv")
df_reviews.head()

In [ ]:
pros = df_reviews['pros_clean_min']
cons = df_reviews['cons_clean_min']

Grouping by company:

In [ ]:
company_pros = df_reviews.groupby('firm')['pros_clean_min'].apply(lambda texts: ' '.join(texts)).reset_index()
company_cons = df_reviews.groupby('firm')['cons_clean_min'].apply(lambda texts: ' '.join(texts)).reset_index()

company_pros.head()

In [ ]:
len(company_pros)

##### LLM for summarization

In [27]:
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="google/pegasus-cnn_dailymail",
    device=-1
)

pytorch_model.bin:  30%|##9       | 682M/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cpu


Example to summarize - Accenture:

In [28]:
company_pros['pros_summary'] = None

company_pros.at[2, 'pros_summary'] = summarizer(
    company_pros.at[2, 'pros_clean_min'],
    max_length=60, 
    min_length=20, 
    do_sample=False
)[0]['summary_text']


In [29]:
company_pros.head()

,firm,pros_clean_min,pros_summary
0,ASOS,"Informal environment and a lot of holiday Perks and good environment great benefits, diverse and international team You get to go home. Eventually.",None
1,AXA UK,"Forward thinking, people centred organisation Family environment, friendly company, interesting work Room to progress Lots of initiatives",None
2,Accenture,"It's very people-centric and there are plenty of career advancement and personal development opportunities Good medical insurance benefits, good variety of clients and service offerings Fantastic people challenging environment Sets you up with the basics for the rest of your career People are incredibly collaborative and willing to help you, whatever the problem. Always very intelligent people so its easy to delegate work and expect a good job to be done. Great benefits and I have been able to take flexible working options now I have kids, without compromising my career in Consulting. - It's useful if you don't know what else you want to do as you'll get a good idea of different roles and industries. - If you are willing to play the game, you can have a career here. You get to work with driven and talented people The caliber of people is very high. Good for socials and an inclusive work environment with lots of arranges activities and drinks. Variety of sectors / areas you can get involved in dependent upon interest / skills Easy to get in as a graduate, big clients Opportunities, progression, finance, promotion, big customers Its culture and values are great Its policies make you feel that they really care for you Great company with lots of opportunities and variety Very flexible to manage your time and effort Good name in the market - Incredibly bright and enthusiastic people to work with - Industry-leading graduate programme - Sense of belonging and overall community feel (predominantly from Consultant level and above) - Multinational FTSE 500 clients on their biggest business and technology challenges - Continuous improvement on existing methodologies (focus on Design Thinking and empathy-based decision making) and overall focus on innovation - Thorough internal training portal and budget for external training - Restructured performance system, aimed at fairer representation good pay compared to other tech services companies",It's very people-centric and there are plenty of career advancement and personal development opportunities .<n>The caliber of people is very high. Good for socials and an inclusive work environment .<n>Its culture and values are great. Its policies make you feel that they really care for
3,Accor,"Friendly, Fast pace, Efficient, Challenging, Interesting there where no positives in working there There was literally nothing good. Work like a dog but dont expect a payrise or a promotion even if you have been working there for a few years now.",None
4,Adecco,"Clear progression routes, great collaboration, brilliant corporate framework and support",None


In [ ]:
len(company_pros)

##### Summarization of **pros** per company

In [ ]:
company_pros['pros_summary'] = company_pros['pros_clean_min'].apply(
    lambda text: summarizer(text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']
)

##### Summarization of **cons** per company

In [ ]:
company_cons['cons_summary'] = company_cons['cons_clean_min'].apply(
    lambda text: summarizer(text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']
)

## Ratings - preparation

## Output formatting